# Lecture5.4 重回帰分析（血圧と年齢の関係）
- "Linear regression using the Framingham Heart Study and StatsModels package"より
- https://github.com/RM-Santiago/framingham

収縮期血圧（sBP）と年齢との臨床的な関連性を評価する疫学研究を実施する予定である。
これまでのデータでは、sBPは年齢とともに増加する傾向にあります。つまり、年をとればとるほど、平均sBPは高くなるのです。

この実験は、フラミンガムコホート（最初で最大の疫学的コホートの1つ - https://en.wikipedia.org/wiki/Framingham_Heart_Study ）のデータを活用し、StatsModelsパッケージを使ってPythonでいくつかのモデルを実行する

臨床的な問い:
### 潜在的な臨床的交絡因子を考慮した場合、年齢と平均sBPの間に線形相関があるのか？
選択した交絡因子：性別、教育、喫煙、血圧降下剤摂取、総コレステロール値

* **帰無仮説 (H0)**: sBPと年齢との間に関連はない (considered the chosen confounders);
* **対立仮説 (H1)**: sBPと年齢には関連性がある (considered the chosen confounders).

We are going to extract a publically available version of the Framingham heart study from; https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset/version/1

On this website, you can also consult the coding for the different variables. For example, regarding gender (variable male; male=1 means male and male=2 means female)

### 使用するパッケージのインポート

In [1]:
import numpy as np 
import pandas as pd
import statsmodels.formula.api as smf

# EDA:データ探索

### データ読み込み(githubサイトのcsvファイルを　DataFrameに読み込む）

### データフレーム表示

In [3]:
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


最初の５行だけが表示された

In [4]:
df.shape

(4240, 16)

データは4240行で、16変数(コラム)であることがわかる

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4240 non-null   int64  
 1   age              4240 non-null   int64  
 2   education        4135 non-null   float64
 3   currentSmoker    4240 non-null   int64  
 4   cigsPerDay       4211 non-null   float64
 5   BPMeds           4187 non-null   float64
 6   prevalentStroke  4240 non-null   int64  
 7   prevalentHyp     4240 non-null   int64  
 8   diabetes         4240 non-null   int64  
 9   totChol          4190 non-null   float64
 10  sysBP            4240 non-null   float64
 11  diaBP            4240 non-null   float64
 12  BMI              4221 non-null   float64
 13  heartRate        4239 non-null   float64
 14  glucose          3852 non-null   float64
 15  TenYearCHD       4240 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 530.1 KB


In [6]:
df.describe()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,4240.000000,4240.000000,4135.000000,4240.000000,4211.000000,4187.000000,4240.000000,4240.000000,4240.000000,4190.000000,4240.000000,4240.000000,4221.000000,4239.000000,3852.000000,4240.000000
mean,0.429245,49.580189,1.979444,0.494104,9.005937,0.029615,0.005896,0.310613,0.025708,236.699523,132.354599,82.897759,25.800801,75.878981,81.963655,0.151887
std,0.495027,8.572942,1.019791,0.500024,11.922462,0.169544,0.076569,0.462799,0.158280,44.591284,22.033300,11.910394,4.079840,12.025348,23.954335,0.358953
min,0.000000,32.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,107.000000,83.500000,48.000000,15.540000,44.000000,40.000000,0.000000
25%,0.000000,42.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,206.000000,117.000000,75.000000,23.070000,68.000000,71.000000,0.000000
50%,0.000000,49.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,234.000000,128.000000,82.000000,25.400000,75.000000,78.000000,0.000000
75%,1.000000,56.000000,3.000000,1.000000,20.000000,0.000000,0.000000,1.000000,0.000000,263.000000,144.000000,90.000000,28.040000,83.000000,87.000000,0.000000
max,1.000000,70.000000,4.000000,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,696.000000,295.000000,142.500000,56.800000,143.000000,394.000000,1.000000


**カテゴリー変数： Education

**離散２値変数; male, CurrentSmoker, BPMeds, prevalentStroke, prevalentHyp, diabetes and TenYearCHD. 


### 単回帰の実施

まず、結果/従属変数である**sysBP**と主な従属変数である**age**の間の関連を**StatsModel パッケージ**を活用して検定する

回帰OLS実施と、引数について

**model1 = smf.ols(formula='sysBP ~ age', data=df).fit()**

In [7]:
model1 = smf.ols(formula='sysBP ~ age', data=df).fit()

In [8]:
print(model1.summary()) 

                            OLS Regression Results                            
Dep. Variable:                  sysBP   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     779.0
Date:                Mon, 28 Mar 2022   Prob (F-statistic):          1.58e-157
Time:                        21:45:51   Log-Likelihood:                -18770.
No. Observations:                4240   AIC:                         3.754e+04
Df Residuals:                    4238   BIC:                         3.756e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     82.1420      1.826     44.992      0.0

summaryを読み解くと、

* 左部分には、問題設定が、
   * 目的変数は sysBP
   * 最小二乗法OLSを
   * 患者数は4240人. 

* 右半分には、モデル診断に関する情報が乗っている、
   * R-squared = 0.155,  このモデルによって、全体の16% の変動が説明可能である。The R-squared は 0–1の範囲で, データがどれだけ回帰曲線に近いかを示す。 言い換えれば、結果（応答データ）の変動のうち、何％がモデルによって説明されるかを示すものである;
   * F-statistic value がこのモデルのp値を計算している。すなわち、Prob (F-statistic)が<0.05.

下半分について

出力を一列ずつ読んでいきます。ここでは、我々の結果（sysbp）と我々の独立変数である年齢（1列目）の間の関連を評価しています。これは、1.0128（または係数、2列目）の正の関連を持ち、標準誤差は0. 036（3列目）、t比率（t分布の場合）27.911（4列目）で、p値<0.001（5列目）、ベータ係数の95%信頼区間は、0.94 2（下限、6列目）と1.084（上限、7列目）に対応することがわかります。もっと簡単に言うと

### 年齢とsBPの間には関連がないという帰無仮説を棄却することができる。年齢と収縮期血圧の間に正の（β係数>1.0）関連があり、これは統計的に有意である（p<0.01、95%CIには係数=0.0は含まれない）。

### 重回帰の実施

今度は交絡因子を追加します。言い換えると、1つ以上の独立変数を追加します（関数の"~"の右側）。

In [11]:
model2 = smf.ols(formula='sysBP ~ age + male + education + cigsPerDay + BPMeds + totChol', data=df).fit()

In [12]:
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                  sysBP   R-squared:                       0.213
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     180.3
Date:                Mon, 28 Mar 2022   Prob (F-statistic):          1.24e-203
Time:                        21:46:17   Log-Likelihood:                -17596.
No. Observations:                4007   AIC:                         3.521e+04
Df Residuals:                    4000   BIC:                         3.525e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     81.1446      2.449     33.137      0.0

###  "education" をカテゴリー変数に変換します

education は4つのカテゴリーをもっています

In [13]:
df.education.value_counts()

1.0    1720
2.0    1253
3.0     689
4.0     473
Name: education, dtype: int64

educationがカテゴリ変数であることをmodel式で明記します。

In [14]:
model3 = smf.ols(formula='sysBP ~ age + male + C(education) + cigsPerDay + BPMeds + totChol', data=df).fit()

In [15]:
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                  sysBP   R-squared:                       0.213
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     135.4
Date:                Mon, 28 Mar 2022   Prob (F-statistic):          9.75e-202
Time:                        21:46:39   Log-Likelihood:                -17595.
No. Observations:                4007   AIC:                         3.521e+04
Df Residuals:                    3998   BIC:                         3.526e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              79.0066    

Same model in another style

In [16]:
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sysBP   R-squared:                       0.213
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     135.4
Date:                Mon, 28 Mar 2022   Prob (F-statistic):          9.75e-202
Time:                        21:46:48   Log-Likelihood:                -17595.
No. Observations:                4007   AIC:                         3.521e+04
Df Residuals:                    3998   BIC:                         3.526e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              79.0066      2.416     32.700      0.000      74.270      83.744
C(education)[T.2.0]    -0.8474      0.760     -1.115      0.265      -2.338       0.643
C(education)[T.3.0]    -3.5724      0.907     -3.937      0.000      -5.351      -1.794
C(education)[T.4.0]    -4.5333      1.043     -4.348      0.000      -6.577      -2.489
age                     0.8365      0.040     21.108      0.000       0.759       0.914
male                   -0.0871      0.670     -0.130      0.897      -1.401       1.226
cigsPerDay             -0.0238      0.028     -0.849      0.396      -0.079       0.031
BPMeds                 26.6926      1.848     14.443      0.000      23.069      30.316
totChol                 0.0537      0.007      7.333      0.000       0.039       0.068
==============================================================================
Omnibus:                      652.299   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1444.451
Skew:                           0.945   Prob(JB):                         0.00
Kurtosis:                       5.253   Cond. No.                     1.96e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.96e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

完璧です、それでは、この結果から、最初の臨床的な質問に対して、以下のように答えることができます。

## 収縮期血圧と年齢の間に関連性はありますか？

### はい、年齢と収縮期血圧の間には正の相関があり（ベータ係数[95%CI]：0.84[0.76;0.91]）、関連しうる交絡因子（モデル中の共変量）の影響を考慮しても統計的に有意（p<0.05、信頼区間に0は含まない）であることがわかりました。

### References

 https://www.w3schools.com/python/python_ml_multiple_regression.asp
 Run a regression More info here: https://www.statsmodels.org/stable/glm.html